### Getting WatchList Update

In [209]:
# Main Code File Creation
import requests
import http.client
import json
import hashlib
from datetime import datetime, timedelta, time
from dateutil.relativedelta import relativedelta
import pytz
import time as tm
import pandas as pd

# Global Variables
config_file="config.json"

def getConfig(parameter):
    with open(config_file) as f:
        configparam = json.load(f)
        retValue = configparam[parameter]
        return retValue
    

In [246]:
# # Variables
# interval = '30minute'
# from_date = (datetime.now()-timedelta(1))
# to_date = (datetime.now()-timedelta(0))
# stock_code = 'NIFTY'
# exchange_code = 'NSE'
# product_type = 'Cash'

# Get Symbol Data From ICICI API 
def iciciGetSymDetail(exchange_code = "NSE",stock_code = "NIFTY",product_type = "Cash",interval = "30minute",
                        from_date = (datetime.now()-timedelta(1)),to_date = (datetime.now()-timedelta(0))):
    try:
        i=0
        while i < 10:
            from_date = from_date-timedelta(i)
            if from_date.weekday() not in (5,6) and from_date.strftime('%Y-%m-%d') not in getConfig('HOLIDAY_LIST'):
                break
            i=i+1       
        from_date = from_date.strftime('%Y-%m-%d')+'T00:00:00.000Z'
        to_date = to_date.strftime('%Y-%m-%d')+'T23:59:59.000Z' 
        change = 'N'
        i=0
        df_tick = icici_api.get_historical_data_v2(interval=interval,
                    from_date= from_date,
                    to_date= to_date,
                    stock_code=stock_code,
                    # stock_code='CNXBAN',
                    exchange_code=exchange_code,
                    # exchange_code='NSE',
                    product_type=product_type)

        if df_tick['Status']==200:
            if len(df_tick['Success']) > 0:        
                df_tick = pd.DataFrame(df_tick['Success'])
                return {"status": "SUCCESS", "data": df_tick}
    except Exception as e:
        return {"status": "FAILURE", "data": str(e)}




In [211]:
# ICICI Session update function
def iciciUpdSessToken(icici_session_id):
    app_key = getConfig('ICICI_API_KEY')
    conn = http.client.HTTPSConnection("api.icicidirect.com")
    payload = str({ "SessionToken": str(icici_session_id), "AppKey": str(app_key) })
    headers = {
                "Content-Type": "application/json"
            }
    conn.request("GET", "/breezeapi/api/v1/customerdetails", payload, headers)
    res = conn.getresponse()
    data = res.read()
    data = json.loads(data.decode("utf-8"))

    with open("config.json","r") as f:
        json_data = json.load(f)
    if data['Status'] == 200:
        print(data)
        json_data["ICICI_SESSION_TOKEN"] = data['Success']['session_token']
        json_data["ICICI_API_SESSION"] = icici_session_id

        with open("config.json", "w") as file:
            json.dump(json_data, file, indent=4)

        return{'status':'SUCCESS', 'data':'Session Updated Successfully.'}
    else:
        return{'status':'FAILURE', 'data':data['Error']}
#     except Exception as e:
    print(e)
#         pass


In [236]:
print(iciciUpdSessToken(14640592))
# iciciGetSymDetail(from_date=datetime.now())


{'status': 'FAILURE', 'data': 'Invalid session.'}


In [244]:
from breeze_connect import BreezeConnect
icici_api = BreezeConnect(api_key = getConfig('ICICI_API_KEY'))
icici_api.generate_session(api_secret=getConfig('ICICI_API_SECRET_KEY'), session_token=getConfig('ICICI_API_SESSION'))


In [245]:
# # Variables
interval = '30minute'
from_date = (datetime.now()-timedelta(1))
to_date = (datetime.now()-timedelta(0))
stock_code = 'NIFTY'
exchange_code = 'NSE'
product_type = 'Cash'

try:
    i=0
    while i < 10:
        from_date = from_date-timedelta(i)
        if from_date.weekday() not in (5,6) and from_date.strftime('%Y-%m-%d') not in getConfig('HOLIDAY_LIST'):
            break
        i=i+1       
    from_date = from_date.strftime('%Y-%m-%d')+'T00:00:00.000Z'
    to_date = to_date.strftime('%Y-%m-%d')+'T23:59:59.000Z' 
    change = 'N'
    i=0
    df_tick = icici_api.get_historical_data_v2(interval=interval,
                from_date= from_date,
                to_date= to_date,
                stock_code=stock_code,
                # stock_code='CNXBAN',
                exchange_code=exchange_code,
                # exchange_code='NSE',
                product_type=product_type)

    if df_tick['Status']==200:
        if len(df_tick['Success']) > 0:        
            df_tick = pd.DataFrame(df_tick['Success'])
            return {"status": "SUCCESS", "data": df_tick}
except Exception as e:
    return {"status": "FAILURE", "data": str(e)}

        
print(df_tick)

       close             datetime exchange_code      high       low      open  \
0   19378.95  2023-07-04 09:00:00           NSE  19412.35  19374.60  19406.60   
1   19308.65  2023-07-04 09:30:00           NSE  19385.40  19308.40  19377.85   
2   19326.75  2023-07-04 10:00:00           NSE  19337.75  19300.25  19308.45   
3   19335.40  2023-07-04 10:30:00           NSE  19349.60  19316.50  19325.75   
4   19345.30  2023-07-04 11:00:00           NSE  19350.80  19326.70  19335.20   
5   19349.60  2023-07-04 11:30:00           NSE  19356.20  19333.30  19345.95   
6   19385.50  2023-07-04 12:00:00           NSE  19392.90  19348.65  19348.65   
7   19400.75  2023-07-04 12:30:00           NSE  19401.05  19380.10  19385.75   
8   19432.50  2023-07-04 13:00:00           NSE  19432.50  19383.80  19400.10   
9   19415.75  2023-07-04 13:30:00           NSE  19433.55  19410.70  19431.40   
10  19364.10  2023-07-04 14:00:00           NSE  19418.10  19364.05  19415.30   
11  19387.70  2023-07-04 14:

In [283]:
# customfunctions
import requests
import http.client
import json
import hashlib
from datetime import datetime, timedelta, time
from dateutil.relativedelta import relativedelta
import pytz
import time as tm
import pandas as pd

import os 

live_feed = 'N'
update_flag = 'N'
update_date = datetime.now()
update_icici_session_notify = 'N'
while True:
    try:
        curr_dt = datetime.now()
        print('Iteration Start - {}'.format(curr_dt))
        watchlist = getConfig('STOCK_CODES')
        file_path = 'WatchList.csv'
        wl = pd.DataFrame()
        scrip = pd.DataFrame()

        if os.path.exists(file_path):
            wl = pd.read_csv(file_path)
            if curr_dt.weekday() not in (5,6) and curr_dt.strftime('%Y-%m-%d') not in getConfig('HOLIDAY_LIST') and curr_dt.date() > update_date.date() and curr_dt.time() < time(8,30) and curr_dt.time() < time(16,30) and update_flag == 'N':
                if curr_dt.minute%5== 0:
                    update_flag = 'Y'
                    update_date = curr_dt 
            elif len(wl) != len(watchlist):
                update_flag = 'Y'
        else:
            update_flag = 'Y'
            
        if update_flag == 'Y':
            update_flag = 'N'
            if os.path.exists('icici_scrip.csv')==False:
                csv_url = "https://traderweb.icicidirect.com/Content/File/txtFile/ScripFile/StockScriptNew.csv"
                response = requests.get(csv_url)
                csv_data = response.text
                csv_file = StringIO(csv_data)
                scrip = pd.read_csv(csv_file)
                scrip.to_csv('icici_scrip.csv',index=False)
            else:
                scrip = pd.read_csv('icici_scrip.csv')

            for i in watchlist:
                wl = pd.concat([wl, scrip.loc[scrip['SC']==i].loc[scrip['SM']==i]])
            wl['Date'] = datetime.now().strftime('%Y-%m-%d')
            wl['Time'] = datetime.now().strftime('%H:%M:%S')
            data_list = [] 

            for i in watchlist:
                data = iciciGetSymDetail(from_date=datetime.now(), stock_code=str(i),interval="5minute")

                if data['status'] == 'SUCCESS':            
                    data = data['data']
                    data['date']=data['datetime']
                    data.set_index('datetime', inplace=True)
                    data.index = pd.to_datetime(data.index)
                    data = data[data.index.time >= time(9,15)]
                    data = data.resample('1D').agg({'date':'last','stock_code':'first','open': 'first','high':'max','low':'min','close':'last'})
                    data_list.append(data.iloc[-1])
                else:
                    raise ValueError(data['data'])

            if len(data_list)>0:
                df = pd.DataFrame(data_list)
                df = df.rename(columns={'stock_code':'CD'})
                wl = pd.merge(wl,df, on='CD')
                print(wl)
#                 wl.to_csv('WatchList.csv',index=False)
                tm.sleep(60)        

    except Exception as e:
        err = str(e)
        if 'Invalid User' in err:
            if upd_dt != curr_dt and update_icici_session_notify == 'N':
                send_whatsapp_msg('ICICI Error', err)
                update_icici_session_notify = 'Y'
        print(err)

    tm.sleep(10)
        

Iteration Start - 2023-07-05 18:18:24.051424
File Condition
2023-07-05 18:18:24.051424 - 2023-07-05 18:18:24.051424 - Y
       SC                              SN   EC      SM      SG  \
0   NIFTY                        NIFTY 50  NSE   NIFTY  EQUITY   
1  CNXBAN                      NIFTY BANK  NSE  CNXBAN  EQUITY   
2  NIFFIN  NIFTY FINANCIAL SERVICES INDEX  NSE  NIFFIN  EQUITY   
3  INDVIX      INDIA VIX VOLATALITY INDEX  NSE  INDVIX  EQUITY   

                  TK  LS      CD               NS    TS        Date      Time  \
0           NIFTY 50   1   NIFTY         NIFTY 50  0.01  2023-07-05  18:18:24   
1         NIFTY BANK   1  CNXBAN       NIFTY BANK  0.01  2023-07-05  18:18:24   
2  NIFTY FIN SERVICE   1  NIFFIN  NIFTY FINANCIAL  0.01  2023-07-05  18:18:24   
3          INDIA VIX   1  INDVIX           INDVIX  0.01  2023-07-05  18:18:24   

                  date      open      high       low     close  
0  2023-07-05 09:15:00  19405.95  19421.25  19340.55  19415.30  
1  2023-07-05

KeyboardInterrupt: 

In [281]:
pd.DataFrame(data_list)
# data_list
# pd.read_csv('WatchList.csv')
# csv_data
# iciciGetSymDetail()

,date,stock_code,open,high,low,close
2023-07-05,2023-07-05 09:15:00,NIFTY,19405.95,19421.25,19340.55,19415.30
2023-07-05,2023-07-05 09:15:00,CNXBAN,45169.70,45416.35,45074.75,45197.30
2023-07-05,2023-07-05 09:15:00,NIFFIN,20354.20,20394.75,20204.30,20269.15
2023-07-05,2023-07-05 09:15:00,INDVIX,11.70,12.21,11.57,11.90


# Main Code File, Only update for Final Run

In [66]:
# customfunctions
import requests
import http.client
import json
import hashlib
from datetime import datetime, timedelta, time
from dateutil.relativedelta import relativedelta
import pytz
import time as tm
import pandas as pd

# Global Variables
config_file="config.json"

def getConfig(parameter):
    with open(config_file) as f:
        configparam = json.load(f)
        retValue = configparam[parameter]
        return retValue

# ICICI Session update function
def iciciUpdSessToken(icici_session_id):
    app_key = getConfig('ICICI_API_KEY')
    conn = http.client.HTTPSConnection("api.icicidirect.com")
    payload = str({ "SessionToken": str(icici_session_id), "AppKey": str(app_key) })
    headers = {
                "Content-Type": "application/json"
            }
    conn.request("GET", "/breezeapi/api/v1/customerdetails", payload, headers)
    res = conn.getresponse()
    data = res.read()
    data = json.loads(data.decode("utf-8"))

    with open("config.json","r") as f:
        json_data = json.load(f)
    if data['Status'] == 200:
        json_data["ICICI_SESSION_TOKEN"] = data['Success']['session_token']
        json_data["ICICI_API_SESSION"] = icici_session_id

        with open("config.json", "w") as file:
            json.dump(json_data, file, indent=4)

        return{'status':'SUCCESS', 'data':'Session Updated Successfully.'}
    else:
        return{'status':'FAILURE', 'data':data['Error']}


# Get Symbol Data From ICICI API 
def iciciGetSymDetail(exchange_code = "NSE",stock_code = "NIFTY",product_type = "Cash",interval = "30minute",
                        from_date = (datetime.now()-timedelta(1)),to_date = (datetime.now()-timedelta(0))):
    try:
        i=0
        while i < 10:
            from_date = from_date-timedelta(i)
            if from_date.weekday() not in (5,6) and from_date.strftime('%Y-%m-%d') not in getConfig('HOLIDAY_LIST'):
                break
            i=i+1       
        from_date = from_date.strftime('%Y-%m-%d')+'T00:00:00.000Z'
        to_date = to_date.strftime('%Y-%m-%d')+'T23:59:59.000Z' 
        change = 'N'
        i=0
        df_tick = icici_api.get_historical_data_v2(interval=interval,
                    from_date= from_date,
                    to_date= to_date,
                    stock_code=stock_code,
                    # stock_code='CNXBAN',
                    exchange_code=exchange_code,
                    # exchange_code='NSE',
                    product_type=product_type)

        if df_tick['Status']==200:
            if len(df_tick['Success']) > 0:        
                df_tick = pd.DataFrame(df_tick['Success'])
                return {"status": "SUCCESS", "data": df_tick}
    except Exception as e:
        return {"status": "FAILURE", "data": str(e)}

In [69]:
iciciGetSymDetail()

{'status': 'SUCCESS',
 'data':                datetime stock_code exchange_code product_type expiry_date  \
 0   2023-07-03 08:30:00      NIFTY           NSE         None        None   
 1   2023-07-03 09:00:00      NIFTY           NSE         None        None   
 2   2023-07-03 09:30:00      NIFTY           NSE         None        None   
 3   2023-07-03 10:00:00      NIFTY           NSE         None        None   
 4   2023-07-03 10:30:00      NIFTY           NSE         None        None   
 5   2023-07-03 11:00:00      NIFTY           NSE         None        None   
 6   2023-07-03 11:30:00      NIFTY           NSE         None        None   
 7   2023-07-03 12:00:00      NIFTY           NSE         None        None   
 8   2023-07-03 12:30:00      NIFTY           NSE         None        None   
 9   2023-07-03 13:00:00      NIFTY           NSE         None        None   
 10  2023-07-03 13:30:00      NIFTY           NSE         None        None   
 11  2023-07-03 14:00:00      NIFT

In [68]:
iciciUpdSessToken(14527665)

{'status': 'SUCCESS', 'data': 'Session Updated Successfully.'}